# Word Embeddings in Pytorch

The module that allows you to use embeddings is torch.nn.Embedding, which takes two arguments: the vocabulary size, and the dimensionality of the embeddings.

To index into this table, you must use torch.LongTensor (since the indices are integers, not floats)

In [57]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [6]:
# Mapping from words to indexes
words_to_idx = {'hello': 0, 'world': 1}
embeds = nn.Embedding(2,5) # 2 words in the vocab, 5 dimensionsal embeddings
lookup_tensor = torch.tensor([words_to_idx["hello"]]).long()
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 1.3790,  2.5286,  0.4107, -0.9880, -0.9081]])


## N-Gram Language Modeling

Recall that in an n-gram language model, given a sequence of words *w*, we want to compute:

P(wi|wi−1,wi−2,…,wi−n+1)

Where Wi is the ith words of the sequence.

In this example, we will compute the loss function on some training examples and update the parameters with backprop.

In [14]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# Shakespeare's Sonnet Num. 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [19]:
# We should tokenize the input, but we will ignore that for now
# build a list of tuples. Each tuple is ([word_i - 2, word_i - 1], target_word)

trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
           for i in range(len(test_sentence) - 2)]

# Print the first 3, just to see what they look like:
print(trigrams[:3])

vocab = set(test_sentence)
words_to_idx = {word: i for i, word in enumerate(vocab)}

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


Creating the class for our language model

In [20]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [27]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10000):
    total_loss = 0
    
    for context, target in trigrams:
        # Step 1) Prepare the inputs to be passed to the model, so turn the words
        # into integer indices and wrap them in long tensors
        context_idxs = torch.tensor([words_to_idx[w] for w in context]).long()
    
        # Step 2) Recall that torch "accumulates" gradients. Before passing
        # in a new instance, we need to zero out the gradients from the old
        # instance
        model.zero_grad()
    
        # Step 3) Run the forward pass, getting the log probabilities over
        # next words
        log_probs = model(context_idxs)
    
        # Step 4) Compute your loss function. (Again, Torch want the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([words_to_idx[target]]).long())
    
        # Step 5) Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()
    
        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)

print(losses[-1])

3.1448485551809426


We can see our loss value decreasing over every iteration!

## Exercise: Computing Word Embeddings: Continious Bag-of-Words

In [67]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_DIM = 10
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for i, word in enumerate(vocab)}

for i, word in enumerate(vocab):
    word_to_ix[word] = i
    ix_to_word[i] = word

data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

# _______________________________________

# Continious Bag of Words Model

class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs
    
    def get_word_embedding(self, word):
        word = torch.tensor([word_to_ix[word]]).long()
        return self.embeddings(word).view(1,-1)


# __________________________________

# Helper Functions

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

def get_index_of_max(input):
    index = 0
    for i in range(1, len(input)):
        if input[i] > input[index]:
            index = i 
    return index

def get_max_prob_result(input, ix_to_word):
    return ix_to_word[get_index_of_max(input)]

# ______________________________________

# Training

losses = []
loss_function = nn.NLLLoss()
model = CBOW(vocab_size, EMBEDDING_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(500):
    total_loss = 0
    
    for context, target in data:
        # Step 1: Prep Inputs
        context_idxs = make_context_vector(context, word_to_ix)
        
        # Step 2: Reset Gradient
        model.zero_grad()
        
        # Step 3: Forward Pass
        log_probs = model(context_idxs)
        
        # Step 4: Compute Loss
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]]).long())
        
        # Step 5: Backward Pass, Update Gradient
        loss.backward()
        optimizer.step()
        
        # Tracking Loss
        total_loss += loss.item()
    losses.append(total_loss)
print(losses[-1])

# _________________________________

# Testing

context = ['of','a','is', 'directed']
context_vector = make_context_vector(context, word_to_ix)
test = model(context_vector).data.numpy()

print("Raw text: {}\n".format(' '.join(raw_text)))
print("Context: {}\n".format(context))
print('Prediction: {}'.format(get_max_prob_result(test[0], ix_to_word)))

2.7819302808493376
Raw text: We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Context: ['of', 'a', 'is', 'directed']

Prediction: process
